### Importing Libraries

In [2]:
#taking care of all the imports
import pandas as pd
import numpy as np

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import requests #for getting webpage data

!conda install -c conda-forge beautifulsoup4 --yes



from bs4 import BeautifulSoup #for web-scraping

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    soupsieve-2.0.1            |   py36h9f0ad1d_0          56 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         219 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.1-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsieve-2.0.1-py36h9f0ad1d_0



beautifulsoup4-4.9.1 | 163 KB    | ################################

In [3]:

#getting the html page
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_text=requests.get(url)

soup=BeautifulSoup(html_text.text,'html.parser')

### Converting data to Data frame

In [4]:
toronto_base=pd.DataFrame()

#marker to avoid getting the first header data
skip_head=True

#looking for row tags
for tag in soup.find_all('tr'):
    #marker to tell which piece of data we should be looking at
    i=1
    
    #iterate through children within the table row
    for ch in tag.children:
        #skip if this is part of the table header or if it's not a table entry
        if(ch.name!=None and skip_head==False):
            if(i==1):  #then this is the Postal Code
                postcode=ch.string
                i=i+1
            elif(i==2):  #then this is the Borough
                borough=ch.string
                i=i+1
            elif(i==3):  #then this is the Neighborhood
                neigh=ch.string
                if(neigh!='Not assigned\n'):  #then need to dig a little deeper into the hyperlink tag to get the data
                    if(ch.a!=None):
                        neigh=ch.a.string
                
                i=i+1
            
    #re-set marker 
    i=1
    
    if(skip_head==False):  #then this is not the first time cycling through
        if(borough!='Not assigned'):  #using this to screen out postal codes that aren't assigned
            #adding the row to the dataframe
            toronto_base=toronto_base.append([[postcode,borough,neigh]])
            
    #after 1st time through, this marker now goes false now that we've skipped the table header
    skip_head=False
    
#add in column headers
toronto_base.columns=('PostalCode','Borough','Neighborhood')
toronto_base.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
0,M2A\n,Not assigned\n,Not assigned\n
0,M3A\n,North York\n,Parkwoods\n
0,M4A\n,North York\n,Victoria Village\n
0,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


### Data Preprocessing

In [5]:
#create the new dataframe with the correct headers
toronto_final=pd.DataFrame(columns=('PostalCode','Borough','Neighborhood'))

#getting one copy of each of the Postal Codes and putting them in the new dataframe
toronto_final['PostalCode']=toronto_base['PostalCode'].unique()
#sorting alphabetically and eliminating the 'None' Postal Code
toronto_final=toronto_final.sort_values('PostalCode')[:-1]

#resetting the index and dropping the old index column
toronto_final.reset_index(inplace=True)
toronto_final.drop(['index'],axis=1,inplace=True)
toronto_final.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,NaN,NaN
1,M1B\n,NaN,NaN
2,M1C\n,NaN,NaN
3,M1E\n,NaN,NaN
4,M1G\n,NaN,NaN


In [6]:

#iterating through each Postal Code
for i in range(0,toronto_final.shape[0]-1):
    #getting the corresponding Borough from the original dataframe and copying it into the new dataframe
    temp=toronto_base.loc[lambda df: df.PostalCode==toronto_final.loc[i]['PostalCode']]['Borough'].unique()
    toronto_final.loc[i]['Borough']=temp[0]
    
    #getting the list of neighborhoods from the original dataframe
    temp=toronto_base.loc[lambda df: df.PostalCode==toronto_final.loc[i]['PostalCode']]['Neighborhood'].unique()
    s='' #creating a new string object
    
    #iterating through the neighborhood list and adding them into the string object
    for j in range(0,len(temp)):
        if(temp[j][-1]=='\n'): #then need to eliminate \n at the end            
            temp[j]=temp[j][:-1]
        if(temp[j]=='Not assigned'): #then there's no Neighborhood name and need to copy the Borough name
            temp[j]=toronto_final.loc[i]['Borough']
        if(s==''):  #then it's the first neighborhood entered
            s=temp[j]
        else:  #there's already neighborhoods in the list and we need a comma
            s=s+", "+temp[j]
    #put the list of neighborhoods into the new dataframe
    toronto_final.loc[i]['Neighborhood']=s
toronto_final.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M1B\n,Scarborough\n,"Malvern, Rouge"
2,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
3,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
4,M1G\n,Scarborough\n,Woburn


In [8]:
print(toronto_final.shape)

(180, 3)
